In [1]:
import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
import numpy as np
from shapely import wkt
from shapely.geometry import Point

In [3]:
war_fires_build_count_set_1 = "../data/raw/war_fires_build_count_set_full.csv"
geo_warfires_df_utm = pd.read_csv(war_fires_build_count_set_1)

In [4]:
geo_warfires_df_utm

,id_w_time,LATITUDE,LONGITUDE,id,x,y,year,time_of_year,ACQ_TIME,date,...,war_fire,sustained_excess,id_big,length_of_war_fire_area,war_fire_restrictive,state,geometry,geometry_buff,building_count,Unnamed: 0
0,28.7-45.6-239-2023,45.55085,28.69054,28.7-45.6,28.7,45.6,2023,239,1104,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329355.658988406 3319152.759123723),"POLYGON ((2329855.658988406 3319152.759123723,...",0,NaN
1,28.7-45.6-239-2023,45.55459,28.69310,28.7-45.6,28.7,45.6,2023,239,1015,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329684.345463021 3319504.96926707),"POLYGON ((2330184.345463021 3319504.96926707, ...",0,NaN
2,28.7-45.6-239-2023,45.55137,28.69108,28.7-45.6,28.7,45.6,2023,239,1155,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329397.980036283 3319222.69775088),"POLYGON ((2329897.980036283 3319222.69775088, ...",0,NaN
3,28.7-45.6-239-2023,45.55492,28.69486,28.7-45.6,28.7,45.6,2023,239,1104,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2329685.16681294 3319710.821818586),"POLYGON ((2330185.16681294 3319710.821818586, ...",0,NaN
4,28.7-45.6-239-2023,45.55830,28.69397,28.7-45.6,28.7,45.6,2023,239,1104,2023-08-27,...,1,1,29-46,531,1,Odesa Oblast,POINT (2330041.047240355 3319664.177652),"POLYGON ((2330541.047240355 3319664.177652, 23...",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39805,40-49.3-128-2023,49.26086,39.99496,40-49.3,40.0,49.3,2023,128,9,2023-05-08,...,1,1,40-49,164,0,Luhansk Oblast,POINT (2407784.855758102 4672156.063601223),"POLYGON ((2408284.855758102 4672156.063601223,...",0,42961.0
39806,40-49.3-128-2023,49.26150,39.99405,40-49.3,40.0,49.3,2023,128,9,2023-05-08,...,1,1,40-49,164,0,Luhansk Oblast,POINT (2407866.943128384 4672068.512588453),"POLYGON ((2408366.943128384 4672068.512588453,...",0,42962.0
39807,40-49.3-128-2023,49.26263,39.98810,40-49.3,40.0,49.3,2023,128,9,2023-05-08,...,1,1,40-49,164,1,Luhansk Oblast,POINT (2408140.1125633 4671426.694026874),"POLYGON ((2408640.1125633 4671426.694026874, 2...",0,42963.0
39808,40-49.3-187-2023,49.27455,40.02201,40-49.3,40.0,49.3,2023,187,1041,2023-07-06,...,1,1,40-49,164,1,Luhansk Oblast,POINT (2408166.684017006 4675501.439766358),"POLYGON ((2408666.684017006 4675501.439766358,...",0,42964.0


### Numbers of building that are affected by war fire (using Intersection of war_fires and buildings)

In [5]:
geo_warfires_df_utm['building_count'].unique()

array([  0,  21,  47,  40,   9,  17,  42,  45,  41,  10,  26,  27,  16,
         1,  25,  15,   2,   3,   8,   5,   4,   6,   7,  37,  29,  72,
        23,  61,  94, 135,  51, 123, 122, 173, 111, 180, 425, 237, 269,
       181,  56, 199, 190, 253, 152, 345,  18,  20,  36,  39,  62,  65,
        13,  90,  80,  33,  19,  82,  28,  74,  44, 150,  35,  64,  69,
        66,  31,  49,  11,  14, 333, 369, 176,  86,  91, 256,  93, 136,
       138,  71, 128, 204, 309,  63,  52, 362,  89,  67, 155, 126, 216,
       320,  12,  22,  96, 104, 142,  59, 205,  92,  34,  32,  46,  38,
       167, 148, 129, 157, 178, 137, 212, 209, 224, 213, 202, 222, 187,
       221, 154, 197, 211, 130, 198, 186, 189, 214, 177, 141, 230, 127,
       133, 188,  84, 159, 169, 238,  77, 210, 223, 158, 117, 114, 170,
       121, 143, 165, 219, 119, 193,  24,  30,  43, 109,  53,  48,  83,
        58,  50,  54,  78,  68, 191, 273, 196, 103, 405, 149, 252, 147,
       164,  81,  99,  97, 312, 300, 112, 120, 396, 279, 734, 71

### Showing building_count based on geometry

In [8]:
import pandas as pd
import folium


# Filter out rows where building_count is not equal to 0
filtered_data = geo_warfires_df_utm[geo_warfires_df_utm['building_count'] != 0]

n = folium.Map(location=[filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()], zoom_start=10)

for index, row in filtered_data.iterrows():
    folium.Marker(
        [row['LATITUDE'], row['LONGITUDE']],
        popup=f"Building Count: {row['building_count']}"
    ).add_to(n)

n.save("../data/raw/ukr_warfire_build_count_markers2.html")


### War fires with buff and number of buildings are affected

In [7]:
import pandas as pd
import folium


# Filter out rows where building_count is greater than 0 and based on geometry_buff
filtered_data = geo_warfires_df_utm[(geo_warfires_df_utm['building_count'] > 0) & (geo_warfires_df_utm['geometry_buff'].notnull())]

m = folium.Map(location=[filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()], zoom_start=10)

for index, row in filtered_data.iterrows():
    folium.Circle(
        radius=10,  # Adjust the radius of the circle as needed
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=f"Building Count: {row['building_count']}",
        color='crimson',  # Color of the circle outline
        fill=True,
        fill_color='crimson'  # Fill color of the circle
    ).add_to(m)

m.save("../data/raw/ukr_warfire_build_count_markers_with_geobuff.html")


In [7]:
m.save("../data/raw/ukr_warfire_build_count.html")

In [9]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [12]:
filtered_data['geometry_wkt2'] = filtered_data['geometry_buff'].apply(lambda x: loads(x) if x is not None else None)

/var/folders/6m/0w40n7p11jd2c0112ngdmyx40000gn/T/ipykernel_1422/1686159841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['geometry_wkt2'] = filtered_data['geometry_buff'].apply(lambda x: loads(x) if x is not None else None)


In [13]:
gdf = GeoDataFrame(filtered_data, crs="EPSG:4326", geometry='geometry_wkt2')

In [17]:
ukr_map = folium.Map(location=[filtered_data['LATITUDE'].mean(), filtered_data['LONGITUDE'].mean()], zoom_start=10)


In [18]:
for _, r in gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry_wkt2"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})
    folium.Popup(r["building_count"]).add_to(geo_j)
    geo_j.add_to(ukr_map)
ukr_map.save("../data/raw/ukr_warfire_build_count_markers_with_geobuff.html")